In [1]:
from sys import maxsize

import numpy as np
import itertools
import math
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit
#from qiskit import Aer

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
#from qiskit.providers.aer import QasmSimulator
import qiskit_aer.noise as noise
from qiskit.circuit.library import UnitaryGate

#My imports
from coolingUnitary import CoolingUnitary
from mirrorProtocol import MirrorProtocol
from pairingPartnerAlgorithm import PairingPartnerAlgorithm
from minimalWorkProtocol import MinimalWorkProtocol
from utils import *

In [2]:
backendName = "ibm_osaka"

In [3]:
service = QiskitRuntimeService()
#sim_backend = service.backend(backendName)
qasm_sim = AerSimulator()

In [4]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

""" qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.barrier()
circuit.x(qreg_q[0])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.x(qreg_q[0])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[2], qreg_q[1], qreg_q[0])
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.x(qreg_q[0]) """
# @columns [1,1,1,2,3,4,5,5,5,6,6,7,8,8,9,9,9,10,10,11,12,12,13,13,13,14,14,15,16,16,17,17,17,18,19,20]

" qreg_q = QuantumRegister(3, 'q')\ncreg_c = ClassicalRegister(3, 'c')\ncircuit = QuantumCircuit(qreg_q, creg_c)\n\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])\ncircuit.x(qreg_q[0])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[1])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[2], qreg_q[1], qreg_q[0])\ncircuit.x(qreg_q[1])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])\ncircuit.x(qreg_q[0]) "

In [5]:
""" circuit.draw('mpl') """

" circuit.draw('mpl') "

In [6]:
def permutationToCircuit(l):
    if(len(l) > 0):
        numQubit = len(l[0][0])
        qreg_q = QuantumRegister(numQubit, 'q')
        creg_c = ClassicalRegister(numQubit, 'c')
        finalCircuit = QuantumCircuit(qreg_q, creg_c)
        finalCircuit.barrier()
        for i in range(len(l)):
            #stateIn = l[i][0]
            #print(l[i][0])
            stateIn = l[i][0][::-1]
            stack = []
            for j in range(numQubit-1,-1,-1):
                #The reason we use opposite is because how Qiskit handles the states
                #The qubit we want to cool down is the n-th one
                opposite = numQubit - j - 1            
                circuit = QuantumCircuit(qreg_q)
                if(l[i][0][j] != l[i][1][j]):
                    
                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                    for z in range(len(stateIn)):
                        if(z != opposite):
                            if(stateIn[z] == '0'):
                                circuit.x(z)
                    
                    #MCX Gate with target in the (numQubit - j - 1) position and control in the rest of the qubits
                    rangeOfMCX = list(range(0,opposite)) + list(range(opposite + 1,numQubit))
                    #print(rangeOfMCX)
                    #print(l[i][0][j] + " " + l[i][1][j])
                    circuit.mcx(list(rangeOfMCX),opposite)

                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state             
                    for z in range(len(stateIn)):
                        if(z != opposite):
                            if(stateIn[z] == '0'):
                                circuit.x(z)

                    #Change the state in to match the state after applying the gate              
                    if(stateIn[opposite] == '0'):
                        stateIn = stateIn[:opposite] + '1' + stateIn[opposite + 1:]
                    else:
                        stateIn = stateIn[:opposite] + '0' + stateIn[opposite + 1:]


                    circuit.barrier()

                    stack.append(circuit)
                    finalCircuit = finalCircuit.compose(circuit)
                    
                

           
            #Using the stack we create the uncomputation circuit
            #We discard the first element since we don't need it for the uncompuation
            stack.pop()
            while(len(stack)):
                finalCircuit = finalCircuit.compose(stack.pop())

    return finalCircuit

In [7]:
def permutationToCircuit2(l,numQubit):

    CoolingUnitary._checkInputParameters(CoolingUnitary,numQubit,l)
    #CoolingUnitary.printSwapList()

    qreg_q = QuantumRegister(numQubit, 'q')
    creg_c = ClassicalRegister(numQubit, 'c')
    finalCircuit = QuantumCircuit(qreg_q, creg_c)
    finalCircuit.barrier()
    for i in range(len(l)):
        #stateIn = l[i][0]
        #print(l[i][0])
        stateIn = integerToBinary(l[i][0],numQubit)[::-1]
        stack = []
        for j in range(numQubit-1,-1,-1):
            #The reason we use opposite is because how Qiskit handles the states
            #The qubit we want to cool down is the n-th one
            opposite = numQubit - j - 1            
            circuit = QuantumCircuit(qreg_q)
            if(integerToBinary(l[i][0],numQubit)[j] != integerToBinary(l[i][1],numQubit)[j]):
                
                #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                for z in range(len(stateIn)):
                    if(z != opposite):
                        if(stateIn[z] == '0'):
                            circuit.x(z)
                
                #MCX Gate with target in the (numQubit - j - 1) position and control in the rest of the qubits
                rangeOfMCX = list(range(0,opposite)) + list(range(opposite + 1,numQubit))
                #print(rangeOfMCX)
                #print(l[i][0][j] + " " + l[i][1][j])
                circuit.mcx(list(rangeOfMCX),opposite)

                #Add X gates when the qubit is in the state 0 so all control are in the 1 state             
                for z in range(len(stateIn)):
                    if(z != opposite):
                        if(stateIn[z] == '0'):
                            circuit.x(z)

                #Change the state in to match the state after applying the gate              
                if(stateIn[opposite] == '0'):
                    stateIn = stateIn[:opposite] + '1' + stateIn[opposite + 1:]
                else:
                    stateIn = stateIn[:opposite] + '0' + stateIn[opposite + 1:]


                circuit.barrier()

                stack.append(circuit)
                finalCircuit = finalCircuit.compose(circuit)
    
        #Using the stack we create the uncomputation circuit
        #We discard the first element since we don't need it for the uncompuation
        stack.pop()
        while(len(stack)):
            finalCircuit = finalCircuit.compose(stack.pop())

    return finalCircuit

In [8]:
def permutationListToCircuit(l,numQubit):

    CoolingUnitary._checkInputParameters(CoolingUnitary,numQubit,l)
    #CoolingUnitary.printSwapList()

    qreg_q = QuantumRegister(numQubit, 'q')
    creg_c = ClassicalRegister(numQubit, 'c')
    finalCircuit = QuantumCircuit(qreg_q, creg_c)
    finalCircuit.barrier()
    for i in range(len(l)):
        #stateIn = l[i][0]
        #print(l[i][0])
        stateIn = integerToBinary(l[i][len(l[i])-2],numQubit)[::-1]
        stack = []
        for t in range(len(l[i])-2,-1,-1):
            #print(stateIn)
            for j in range(numQubit-1,-1,-1):
                #The reason we use opposite is because how Qiskit handles the states
                #The qubit we want to cool down is the n-th one
                opposite = numQubit - j - 1            
                circuit = QuantumCircuit(qreg_q)
                if(integerToBinary(l[i][t],numQubit)[j] != integerToBinary(l[i][t-1],numQubit)[j]):
                    
                    
                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                    xList = []
                    for z in range(len(stateIn)):
                        if(z != opposite):
                            if(stateIn[z] == '0'):
                                xList.append[z]

                    circuit.x(xList)
                    #MCX Gate with target in the (numQubit - j - 1) position and control in the rest of the qubits
                    rangeOfMCX = list(range(0,opposite)) + list(range(opposite + 1,numQubit))
                    #print(rangeOfMCX)
                    #print(l[i][0][j] + " " + l[i][1][j])
                    circuit.mcx(list(rangeOfMCX),opposite)

                    circuit.x(xList)

                    #Change the state in to match the state after applying the gate              
                    if(stateIn[opposite] == '0'):
                        stateIn = stateIn[:opposite] + '1' + stateIn[opposite + 1:]
                    else:
                        stateIn = stateIn[:opposite] + '0' + stateIn[opposite + 1:]


                    circuit.barrier()

                    stack.append(circuit)
                    finalCircuit = finalCircuit.compose(circuit)

            #Using the stack we create the uncomputation circuit
            #We discard the first element since we don't need it for the uncompuation
            stack.pop()
            while(len(stack)):
                finalCircuit = finalCircuit.compose(stack.pop())

    return finalCircuit

In [9]:
def testCircuit(circuit):
    n = circuit.num_qubits
    num_circs = 2**n
    circuits = []
    q_regs = QuantumRegister(n, 'q')
    c_regs = ClassicalRegister(n, 'c')
    #make circuits
    for i in range(num_circs):
        total_circ = QuantumCircuit(q_regs, c_regs)
        #prepare initial state
        bitstring = format(i,'b').zfill(n)
        idx = 0
        for bit in bitstring[::-1]:
        #for bit in bitstring:
            if (bit == '1'):
                total_circ.x(idx)
            idx += 1
        total_circ.barrier()
        #add cooling circuits
        #total_circ.compose(circuit, qubits=[q_regs[2],q_regs[1],q_regs[0]], inplace=True)
        total_circ = total_circ.compose(circuit)
        #total_circ.compose(cooling_circuit, qubits=[q_regs[0],q_regs[1],q_regs[2],q_regs[3],q_regs[4]], inplace=True)
        #add measurement
        total_circ.barrier()
        total_circ.measure(list(range(n)), c_regs)
        #total_circ.measure(0, c_regs)
        #transpiledCircuit = qk.transpile(total_circ, backend=sim_backend,optimization_level=3)
        circuits.append(total_circ)
    
    counts = []

    numOfDigits = len(str(num_circs))
    print(format(0).zfill(numOfDigits) + " | ",end="")
    for i in range(n-1,0,-1):
        print(i,end="")
    print(0)
    print("---------------------")
    for i in range(num_circs):
        result = qasm_sim.run(circuits[i],shots = 1).result()
        counts.append(result.get_counts())
        #print(i, list(counts[i])[0])
        if(format(i,'b').zfill(n) == list(counts[i])[0]):
            print(format(i).zfill(numOfDigits) + " | " + format(i,'b').zfill(n) + " --> " + str(list(counts[i])[0]))
        else:
            print(format(i).zfill(numOfDigits) + " | " + format(i,'b').zfill(n) + " --> " + str(list(counts[i])[0]) + " (*)")
    #return circuits

In [10]:
l = [[3,4]]
n = 3
c = permutationListToCircuit(l,5)

testCircuit(c)
c.draw(fold=maxsize)

00 | 43210
---------------------
00 | 00000 --> 00000
01 | 00001 --> 00001
02 | 00010 --> 00010
03 | 00011 --> 00100 (*)
04 | 00100 --> 00011 (*)
05 | 00101 --> 00101
06 | 00110 --> 00110
07 | 00111 --> 00111
08 | 01000 --> 01000
09 | 01001 --> 01001
10 | 01010 --> 01010
11 | 01011 --> 01011
12 | 01100 --> 01100
13 | 01101 --> 01101
14 | 01110 --> 01110
15 | 01111 --> 01111
16 | 10000 --> 10000
17 | 10001 --> 10001
18 | 10010 --> 10010
19 | 10011 --> 10011
20 | 10100 --> 10100
21 | 10101 --> 10101
22 | 10110 --> 10110
23 | 10111 --> 10111
24 | 11000 --> 11000
25 | 11001 --> 11001
26 | 11010 --> 11010
27 | 11011 --> 11011
28 | 11100 --> 11100
29 | 11101 --> 11101
30 | 11110 --> 11110
31 | 11111 --> 11111


░      ┌───┐      ░ ┌───┐     ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐     ┌───┐ ░      ┌───┐      ░ 
q_0: ─░──────┤ X ├──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░──────┤ X ├──────░─
      ░      └─┬─┘      ░ └───┘┌─┴─┐└───┘ ░ ├───┤  │  ├───┤ ░ └───┘┌─┴─┐└───┘ ░      └─┬─┘      ░ 
q_1: ─░────────■────────░──────┤ X ├──────░─┤ X ├──■──┤ X ├─░──────┤ X ├──────░────────■────────░─
      ░ ┌───┐  │  ┌───┐ ░ ┌───┐└─┬─┘┌───┐ ░ └───┘┌─┴─┐└───┘ ░ ┌───┐└─┬─┘┌───┐ ░ ┌───┐  │  ┌───┐ ░ 
q_2: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░──────┤ X ├──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ ┌───┐└─┬─┘┌───┐ ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ 
q_3: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ 
q_4: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ └───┘     └───┘ ░ └───┘     └───┘ ░ └───┘     └───┘ ░ └───┘     └───┘ ░ └───┘     └───┘ ░ 
c: 5/═════════════════════════════════════════════════════════════════════════════════════════════

In [13]:
circu = QuantumCircuit(3)
circu.x([0,1,2])
circu.draw(fold=maxsize)

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘

In [95]:
def coolingUnitaryToCircuit2(m):
    # Check if the matrix is unitary
    if(is_unitary(m) == False):
        raise ValueError("Not an unitary Matrix")
    
    dictionary = {}
    numberOfStates = len(m)
    numQubits = int(math.log2(numberOfStates))
    # 1 column with 2 ** NumQubits 
    state = np.zeros((numberOfStates,1))
    
    for i in range(numberOfStates):
        # i = 0,  State 00 = [1 0 0 0] 
        # i = 1,  State 01 = [0 1 0 0] 

        #Application of the unitary
        state[i,0] = 1 
        matrix = m.dot(state) 
        indices = np.where(matrix == 1)

        #Probably not necessary
        if len(indices[0]) > 1:
            raise ValueError("Error")
            
        outputState = int(indices[0][0])
        #State -> State dictionary
        if(outputState != i):
            if i not in dictionary:
                if outputState not in dictionary:
                    dictionary[i] = [outputState]
                else:
                    dictionary[outputState].append(i)
                    #for i in range(len(dictionary[outputState])):
                    #    if dictionary[outputState][i] != i:
                    #        dictionary[i] = outputState
        #print(state)
        #print(matrix)

        # Return to all zero matrix
        state[i,0] = 0 

    permutationList = []
    for key, value in dictionary.items():
        l = []
        l.append(key)

    
    return dictionary

In [220]:
def coolingUnitaryToPermutationList(m):
    """
    Parameters:
        CoolingUnitary (numpy.ndarray)
    Return:
        Permutation List (list)
    """
    # Check if the matrix is unitary
    if(is_unitary(m) == False):
        raise ValueError("Not an unitary Matrix")
    
    statesInSwapCycle = set()
    numberOfStates = len(m)
    #print(m)
    permutationsList = []
    for i in range(numberOfStates):
        index = i
        #If the state is NOT swapped with itself
        if(m[index][index] != 1):
            #Check if the state is NOT already inside of a swap cycle
            if(index not in statesInSwapCycle):
                l = [index]
                #print(index)
                statesInSwapCycle.add(index)
                nextIndex = -1

                #Find this state is swapped to
                for j in range(len(m[i])):
                    if(m[j][i] == 1):
                        nextIndex = j
                        break
                #Cycle until we return to the starting state
                while(index != nextIndex):
                    #print(nextIndex)
                    l.append(nextIndex)
                    statesInSwapCycle.add(nextIndex)
                    for j in range(len(m[nextIndex])):
                        if(m[j][nextIndex] == 1):
                            nextIndex = j
                            break 
                permutationsList.append(l)

    return permutationsList
        

In [85]:
n = 5
a = MinimalWorkProtocol(n,0.01)
#checkUnitary(a)
cooling_circuit = QuantumCircuit(n)
cooling_circuit.append(UnitaryGate(a),range(n))

testCircuit(cooling_circuit)

lista = [[1,0,2],[3,4]]
obj = CoolingUnitary(3,lista)
print(coolingUnitaryToPermutationList(a))


00 | 43210
---------------------
00 | 00000 --> 00000
01 | 00001 --> 00001
02 | 00010 --> 00010
03 | 00011 --> 10000 (*)
04 | 00100 --> 00100
05 | 00101 --> 00101
06 | 00110 --> 00110
07 | 00111 --> 11000 (*)
08 | 01000 --> 01000
09 | 01001 --> 01001
10 | 01010 --> 01010
11 | 01011 --> 10100 (*)
12 | 01100 --> 01100
13 | 01101 --> 10010 (*)
14 | 01110 --> 10001 (*)
15 | 01111 --> 00011 (*)
16 | 10000 --> 11100 (*)
17 | 10001 --> 01110 (*)
18 | 10010 --> 01101 (*)
19 | 10011 --> 10011
20 | 10100 --> 01011 (*)
21 | 10101 --> 10101
22 | 10110 --> 10110
23 | 10111 --> 10111
24 | 11000 --> 00111 (*)
25 | 11001 --> 11001
26 | 11010 --> 11010
27 | 11011 --> 11011
28 | 11100 --> 01111 (*)
29 | 11101 --> 11101
30 | 11110 --> 11110
31 | 11111 --> 11111
[[3, 15, 28, 16], [7, 24], [11, 20], [13, 18], [14, 17]]


In [224]:
n = 6
a = MinimalWorkProtocol(n,0.01)
#checkUnitary(a)
cooling_circuit = QuantumCircuit(n)
cooling_circuit.append(UnitaryGate(a),range(n))

testCircuit(cooling_circuit)

00 | 543210
---------------------
00 | 000000 --> 000000
01 | 000001 --> 000001
02 | 000010 --> 000010
03 | 000011 --> 100000 (*)
04 | 000100 --> 000100
05 | 000101 --> 000101
06 | 000110 --> 000110
07 | 000111 --> 110000 (*)
08 | 001000 --> 001000
09 | 001001 --> 001001
10 | 001010 --> 001010
11 | 001011 --> 101000 (*)
12 | 001100 --> 001100
13 | 001101 --> 100100 (*)
14 | 001110 --> 100010 (*)
15 | 001111 --> 000011 (*)
16 | 010000 --> 010000
17 | 010001 --> 010001
18 | 010010 --> 010010
19 | 010011 --> 100001 (*)
20 | 010100 --> 010100
21 | 010101 --> 010101
22 | 010110 --> 010110
23 | 010111 --> 110100 (*)
24 | 011000 --> 011000
25 | 011001 --> 011001
26 | 011010 --> 011010
27 | 011011 --> 110010 (*)
28 | 011100 --> 011100
29 | 011101 --> 110001 (*)
30 | 011110 --> 101100 (*)
31 | 011111 --> 000111 (*)
32 | 100000 --> 111000 (*)
33 | 100001 --> 010011 (*)
34 | 100010 --> 001110 (*)
35 | 100011 --> 100011
36 | 100100 --> 001101 (*)
37 | 100101 --> 100101
38 | 100110 --> 100110
39 | 

In [223]:
n = 6
p = coolingUnitaryToPermutationList(MinimalWorkProtocol(n,0.01))
print(p)
c = permutationToCircuit3(p,n)

testCircuit(c)

[[3, 32, 56, 15], [7, 48, 60, 31], [11, 40], [13, 36], [14, 34], [19, 33], [23, 52], [27, 50], [29, 49], [30, 44]]
00 | 543210
---------------------
00 | 000000 --> 000000
01 | 000001 --> 000001
02 | 000010 --> 000010
03 | 000011 --> 100000 (*)
04 | 000100 --> 000100
05 | 000101 --> 000101
06 | 000110 --> 000110
07 | 000111 --> 110000 (*)
08 | 001000 --> 001000
09 | 001001 --> 001001
10 | 001010 --> 001010
11 | 001011 --> 101000 (*)
12 | 001100 --> 001100
13 | 001101 --> 100100 (*)
14 | 001110 --> 100010 (*)
15 | 001111 --> 000011 (*)
16 | 010000 --> 010000
17 | 010001 --> 010001
18 | 010010 --> 010010
19 | 010011 --> 100001 (*)
20 | 010100 --> 010100
21 | 010101 --> 010101
22 | 010110 --> 010110
23 | 010111 --> 110100 (*)
24 | 011000 --> 011000
25 | 011001 --> 011001
26 | 011010 --> 011010
27 | 011011 --> 110010 (*)
28 | 011100 --> 011100
29 | 011101 --> 110001 (*)
30 | 011110 --> 101100 (*)
31 | 011111 --> 000111 (*)
32 | 100000 --> 111000 (*)
33 | 100001 --> 010011 (*)
34 | 100010 -

In [218]:
lista = [[0,1,3,4],[5,6]]
obj = CoolingUnitary(3,lista)
#checkUnitary(obj)

p = coolingUnitaryToPermutationList(obj)
print(p)

[[0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
0
1
3
4
5
6
[[0, 1, 3, 4], [5, 6]]
